# Bond markets

In [2]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
import plotly.graph_objects as go
np.set_printoptions(suppress=True)

## Term structure from FRED

In [3]:
tseries = ['DGS'+x for x in ['1MO','3MO','1','2','3','5','10','20','30']]
df = pdr(tseries, "fred", start="1929-12-01")
df

,DGS1MO,DGS3MO,DGS1,DGS2,DGS3,DGS5,DGS10,DGS20,DGS30
DATE,,,,,,,,,
1962-01-02,NaN,NaN,3.22,NaN,3.70,3.88,4.06,4.07,NaN
1962-01-03,NaN,NaN,3.24,NaN,3.70,3.87,4.03,4.07,NaN
1962-01-04,NaN,NaN,3.24,NaN,3.69,3.86,3.99,4.06,NaN
1962-01-05,NaN,NaN,3.26,NaN,3.71,3.89,4.02,4.07,NaN
1962-01-08,NaN,NaN,3.31,NaN,3.71,3.91,4.03,4.08,NaN
...,...,...,...,...,...,...,...,...,...
2025-02-06,4.37,4.34,4.19,4.21,4.23,4.28,4.45,4.70,4.65
2025-02-07,4.37,4.35,4.25,4.29,4.31,4.34,4.49,4.75,4.69
2025-02-10,4.38,4.35,4.24,4.28,4.30,4.34,4.51,4.76,4.71


In [4]:
# Convert to monthly
df = df.reset_index()
df['month'] = df.DATE.dt.to_period('M')
df = df.groupby('month').last()
df

,DATE,DGS1MO,DGS3MO,DGS1,DGS2,DGS3,DGS5,DGS10,DGS20,DGS30
month,,,,,,,,,,
1962-01,1962-01-31,NaN,NaN,3.29,NaN,3.81,3.99,4.10,4.13,NaN
1962-02,1962-02-28,NaN,NaN,3.21,NaN,3.53,3.77,4.00,4.10,NaN
1962-03,1962-03-30,NaN,NaN,2.97,NaN,3.39,3.61,3.86,3.98,NaN
1962-04,1962-04-30,NaN,NaN,3.07,NaN,3.47,3.64,3.86,3.91,NaN
1962-05,1962-05-31,NaN,NaN,2.99,NaN,3.36,3.66,3.90,3.94,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-10,2024-10-31,4.76,4.64,4.27,4.16,4.12,4.15,4.28,4.58,4.47
2024-11,2024-11-29,4.76,4.58,4.30,4.13,4.10,4.05,4.18,4.45,4.36
2024-12,2024-12-31,4.40,4.37,4.16,4.25,4.27,4.38,4.58,4.86,4.78


In [5]:
# Two ways to filter on a period index
# df.loc['2000']

df.loc['1992-10']


,1992-10
DATE,1992-10-30 00:00:00
DGS1MO,NaN
DGS3MO,3.03
DGS1,3.54
DGS2,4.4
DGS3,4.98
DGS5,5.9
DGS10,6.8
DGS20,NaN
DGS30,7.63


In [6]:
ttms = [0.0833, 0.25, 1, 2, 3, 5, 10, 20, 30]
months = ['1983-07', '1992-10', '2000-08', '2020-02', ]
yields = df.loc[months[1], tseries]
yields.index = ttms
yields=yields.dropna()
yields

,1992-10
0.25,3.03
1.00,3.54
2.00,4.4
3.00,4.98
5.00,5.9
10.00,6.8
30.00,7.63


In [7]:
fig = go.Figure()
for m in months:
    yields = df.loc[m, tseries]
    yields.index = ttms
    yields=yields.dropna()

    trace= go.Scatter(
                    x=yields.index,
                    y=yields,
                    hovertemplate="<br>" + str(m) +"<br>%{x:.2f} year bond <br>%{y:.1f}%<extra></extra>",
                    mode="lines+markers",
                    name=m)
    fig.add_trace(trace)
fig.update_yaxes(title='Yield (%)',tickformat=".0f")
fig.update_xaxes(title='Maturity', tickformat=".0f")
# fig.update_layout(title='Yield Curve')
fig.show()


In [8]:
# Pull recession data
rec = pdr("USREC", "fred", start="1935-12-01")
starts = rec[(rec.USREC==1) & (rec.USREC.shift()==0)]
ends   = rec[(rec.USREC==1) & (rec.USREC.shift(-1)==0)]
dates = pd.merge(starts.reset_index()['DATE'], ends.reset_index()['DATE'],left_index=True,right_index=True )
dates.columns=['start','end']
dates.start =dates.start.dt.to_period('M')
dates.end   =dates.end.dt.to_period('M')

In [9]:
# Create list of dictionaries for shading
def dict_creator(start,end):
    d = dict(
            type="rect",
            xref="x",
            yref="paper",
            x0=str(start),
            y0=0,
            x1=str(end),
            y1=1,
            fillcolor="DarkGray",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
    return d
dict_list = [dict_creator(dates.loc[i,'start'], dates.loc[i,'end']) for i in dates.index]

In [10]:
fig = go.Figure()
series_to_plot = ['DGS3MO','DGS1','DGS10']

trace1= go.Scatter(x=df.index.astype(str), y=df['DGS1'], mode="lines", name='1-yr Treasury',
                hovertemplate="1-yr Treasury<br>%{x}: %{y:.1f}%<extra></extra>")
fig.add_trace(trace1)

trace10= go.Scatter(x=df.index.astype(str), y=df['DGS10'], mode="lines", name='10-yr Treasury',
                hovertemplate="10-yr Treasury<br>%{x}: %{y:.1f}%<extra></extra>")
fig.add_trace(trace10)

fig.update_yaxes(title='Yield (%)',tickformat=".0f")
fig.update_xaxes(title='Date')
fig.update_layout(xaxis_range=[str(min(df.index)),str(max(df.index))])
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.6))
# add nber recession indicators
fig.update_layout(shapes=dict_list)
fig.show()

## Bootstrapping

In [11]:
import numpy as np
import pandas as pd
A = [ 97.09,      0, 0.5, 100]
B = [ 94.26,      0, 1.0, 100]
C = [943.43,   0.02, 1.5, 1000]
D = [981.41,   0.05, 2.0, 1000]
E = [954.20,   0.04, 2.5, 1000]
F = [972.91,   0.05, 3.0, 1000]
G = [906.55,   0.03, 3.5, 1000]
H = [964.90,   0.05, 4.0, 1000]
bonds = np.row_stack((A, B, C, D, E, F, G, H))

In [12]:
df = pd.DataFrame(bonds,
        columns = ['price', 'coupon_rate', 'maturity','face'],
        index=['A','B','C','D', 'E', 'F', 'G', 'H'])
df['period'] = df.maturity*2
df

,price,coupon_rate,maturity,face,period
A,97.09,0.00,0.5,100.0,1.0
B,94.26,0.00,1.0,100.0,2.0
C,943.43,0.02,1.5,1000.0,3.0
D,981.41,0.05,2.0,1000.0,4.0
E,954.20,0.04,2.5,1000.0,5.0
F,972.91,0.05,3.0,1000.0,6.0
G,906.55,0.03,3.5,1000.0,7.0
H,964.90,0.05,4.0,1000.0,8.0


In [13]:
df=df.reset_index()
df.index = df.index+1
df.columns = ['bond', 'price', 'coupon_rate', 'maturity','face', 'period']
df

,bond,price,coupon_rate,maturity,face,period
1,A,97.09,0.00,0.5,100.0,1.0
2,B,94.26,0.00,1.0,100.0,2.0
3,C,943.43,0.02,1.5,1000.0,3.0
4,D,981.41,0.05,2.0,1000.0,4.0
5,E,954.20,0.04,2.5,1000.0,5.0
6,F,972.91,0.05,3.0,1000.0,6.0
7,G,906.55,0.03,3.5,1000.0,7.0
8,H,964.90,0.05,4.0,1000.0,8.0


In [14]:
# Find z1

spot_rates = []
nop = 2
t = 1
p      = df.loc[t,'price']
face   = df.loc[t,'face']
cr     = df.loc[t,'coupon_rate']
period = df.loc[t,'period']
coupon = face*cr/nop
cf_maturity = face + coupon

z1 = (cf_maturity / p )**(1/period) - 1
spot_rates.append(z1)
z1


0.02997219075084967

In [15]:
# Find z2
t = 2
p      = df.loc[t,'price']
face   = df.loc[t,'face']
cr     = df.loc[t,'coupon_rate']
period = df.loc[t,'period']
coupon = face*cr/nop
cf_maturity = face + coupon

z2 = (cf_maturity / p )**(1/period) - 1
spot_rates.append(z2)
z2



0.029997764907275126

In [16]:
# Find z3

t = 3
p      = df.loc[t,'price']
face   = df.loc[t,'face']
cr     = df.loc[t,'coupon_rate']
period = int(df.loc[t,'period'])
coupon = face*cr/nop
cf_maturity = face + coupon

pv_coupons = 0.0
for i in range(period-1):
    print(i)
    pv_coupons = pv_coupons + coupon / (1+spot_rates[i])**(i+1)
x = p - pv_coupons
z3 = (cf_maturity / x )**(1/period) - 1
spot_rates.append(z3)
z3



0
1


0.0299992853016684

In [17]:
# Find z4
t = 4
p      = df.loc[t,'price']
face   = df.loc[t,'face']
cr     = df.loc[t,'coupon_rate']
period = int(df.loc[t,'period'])
coupon = face*cr/nop
cf_maturity = face + coupon

pv_coupons = 0.0
for i in range(period-1):
    print(i)
    pv_coupons = pv_coupons + coupon / (1+spot_rates[i])**(i+1)
x = p - pv_coupons
z4 = (cf_maturity / x )**(1/period) - 1
spot_rates.append(z4)
z4

0
1
2


0.030001502317938078

In [18]:
# Find z5
t = 5
p      = df.loc[t,'price']
face   = df.loc[t,'face']
cr     = df.loc[t,'coupon_rate']
period = int(df.loc[t,'period'])
coupon = face*cr/nop
cf_maturity = face + coupon

pv_coupons = 0.0
for i in range(period-1):
    print(i)
    pv_coupons = pv_coupons + coupon / (1+spot_rates[i])**(i+1)
x = p - pv_coupons
z5 = (cf_maturity / x )**(1/period) - 1
spot_rates.append(z5)
z5

0
1
2
3


0.030000812108414543

In [19]:
# Find z6
t = 6
p      = df.loc[t,'price']
face   = df.loc[t,'face']
cr     = df.loc[t,'coupon_rate']
period = int(df.loc[t,'period'])
coupon = face*cr/nop
cf_maturity = face + coupon

pv_coupons = 0.0
for i in range(period-1):
    print(i)
    pv_coupons = pv_coupons + coupon / (1+spot_rates[i])**(i+1)
x = p - pv_coupons
z6 = (cf_maturity / x )**(1/period) - 1
spot_rates.append(z6)
z6

0
1
2
3
4


0.030000926589999644

### Find new bond price

In [20]:
##2##
spot_rates.clear()
spot_rates.extend([0.025, 0.026, 0.027, 0.028, 0.029, 0.03, 0.03, 0.03])
cfs = np.array([400, 1400, 300, 1300, 200, 1200, 100, 1100])
discount_factors = np.array([1/(1+spot_rates[i])**(i+1) for i in range(len(spot_rates))])
discount_factors

array([0.97560976, 0.94995991, 0.92318485, 0.89542155, 0.86680843,
       0.83748426, 0.81309151, 0.78940923])

In [21]:
cfs @ discount_factors

5289.193349412445

In [24]:
##333
spot_rates.clear()
spot_rates.extend([0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03])
cfs = np.array([0, 70, 0, 70, 0, 70, 0, 1070])
discount_factors = np.array([1/(1+spot_rates[i])**(i+1) for i in range(len(spot_rates))])
discount_factors

array([0.97087379, 0.94259591, 0.91514166, 0.88848705, 0.86260878,
       0.83748426, 0.81309151, 0.78940923])

In [25]:
cfs @ discount_factors

1031.4675856772278

### Functions to generalize bootstrapping and pricing

In [ ]:
def bootstrap(d, nop):
    '''
    d: a dataset with prices, face value, coupon_rate, period of maturity
    nop: compounding period (ie., 1 = annual, 2=semiannual)
    '''
    spot_rates = []
    for t in d.index:
        p      = d.loc[t,'price']
        face   = d.loc[t,'face']
        cr     = d.loc[t,'coupon_rate']
        period = int(d.loc[t,'period'])
        coupon = face*cr/nop
        cf_maturity = face + coupon

        pv_coupons = 0.0
        for i in range(period-1):
            pv_coupons = pv_coupons + coupon / (1+spot_rates[i])**(i+1)
        x = p - pv_coupons
        z = (cf_maturity / x )**(1/period) - 1
        spot_rates.append(z)
    return spot_rates
spot_curve = bootstrap(df,2)
spot_curve

[0.02564102564102555,
 0.025978352085153977,
 0.028390767355670476,
 0.02882263912764249]

In [ ]:
def spot_pricer(cfs, spot):
    '''
    cfs:  list of cash flows at each period
    spot: list of spot rates at each period
    cfs should be the same length as spot
    '''
    cfs = np.array(cfs)
    discount_factors = np.array([1/(1+spot[i])**(i+1) for i in range(len(spot))])
    return cfs @ discount_factors
spot_pricer([0,100,0,1100], spot_curve)

1076.8171999459971

In [ ]:
# Check that the prices are consistent with the spot curve

# pick a bond
BOND_ID = 'A'
bond_dict = {'A': 1, 'B':2, 'C':3, 'D':4}
def cf_generator(bond_id):
    i = bond_dict[bond_id]
    nop = 2
    face   = df.loc[i,'face']
    cr     = df.loc[i,'coupon_rate']
    period = int(df.loc[i,'period'])
    coupon = face*cr/nop
    cf_maturity = face + coupon
    cfs =  np.zeros(len(spot_curve))
    for i in range(period-1):
        cfs[i] = coupon
    cfs[period-1] = cf_maturity
    print('The bond cash flows for bond ' + bond_id + ' are: ', cfs)
    return cfs
cfs = cf_generator(BOND_ID)

p = spot_pricer(cfs.tolist(),spot_curve)
print('The fair price for ' + BOND_ID + f' is: ${p:,.2f}')


The bond cash flows for bond A are:  [100.   0.   0.   0.]
The fair price for A is: $97.50
